In [1]:
import keyring
from pprint import pprint

api_key_irp = keyring.get_password("irp_api_key", "birdblues")
api_secret_irp = keyring.get_password("irp_api_secret", "birdblues")

api_key_isa = keyring.get_password("isa_api_key", "birdblues")
api_secret_isa = keyring.get_password("isa_api_secret", "birdblues")

api_key_0 = keyring.get_password("api_key_0", "birdblues")
api_secret_0 = keyring.get_password("api_secret_0","birdblues")

api_key_1 = keyring.get_password("api_key_1", "birdblues")
api_secret_1 = keyring.get_password("api_secret_1","birdblues")

api_key_2 = keyring.get_password("api_key_2", "birdblues")
api_secret_2 = keyring.get_password("api_secret_2","birdblues")

# print(f"api_key_irp = {api_key_irp}")
# print(f"api_secret_irp = {api_secret_irp}")
# print(f"api_key_isa = {api_key_isa}")
# print(f"api_secret_isa = {api_secret_isa}")
# print(f"api_key_0 = {api_key_0}")
# print(f"api_secret_0 = {api_secret_0}")
# print(f"api_key_1 = {api_key_1}")
# print(f"api_secret_1 = {api_secret_1}")
# print(f"api_key_2 = {api_key_2}")
# print(f"api_secret_2 = {api_secret_2}")

In [2]:
acc_no_0 = "63981494-01" # 위탁
acc_no_1 = "64267316-22" # 연금저축 1
acc_no_2 = "64012548-22" # 연금저축 2
acc_no_isa = "64043278-01" # ISA
acc_no_irp = "64012548-29" # IR

In [3]:
import keyring

token_0 = keyring.get_password("access_token_0", "birdblues")
token_1 = keyring.get_password("access_token_1", "birdblues")
token_2 = keyring.get_password("access_token_2", "birdblues")
token_3 = keyring.get_password("access_token_3", "birdblues")
token_4 = keyring.get_password("access_token_4", "birdblues")

# print(f"token_0 = {token_0}")
# print(f"token_1 = {token_1}")
# print(f"token_2 = {token_2}")
# print(f"token_3 = {token_3}")
# print(f"token_4 = {token_4}")

In [34]:
import requests
import pandas as pd

pd.options.display.float_format = '{:.2f}'.format

column_mapping = {
    'trad_dt': '거래일',
    'pdno': '코드',
    'prdt_name': '상품명',
    'trad_dvsn_name': '거래구분',
    'loan_dt': '대출일',
    'hldg_qty': '잔고',
    'pchs_unpr': '매입가격',
    'buy_qty': '매수수량',
    'buy_amt': '매수금액',
    'sll_pric': '매도가격',
    'sll_qty': '매도수량',
    'sll_amt': '매도금액',
    'rlzt_pfls': '실현손익',
    'pfls_rt': '손익률',
    'fee': '수수료',
    'tl_tax': '세금',
    'loan_int': '대출이자'
}

cols_to_convert = ['잔고', '매입가격', '매수수량', '매수금액', '매도가격', '매도수량', '매도금액', '실현손익', '손익률', '수수료', '세금', '대출이자']

def get_trade_profit(access_token, app_key, app_secret, acc_no):
    
    output1_list = []
    
    url_base = "https://openapi.koreainvestment.com:9443"
    path = "/uapi/domestic-stock/v1/trading/inquire-period-trade-profit"
    url = f"{url_base}/{path}"

    headers = {
        "Content-Type": "application/json; charset=utf-8",
        "authorization": f"Bearer {access_token}",
        "appKey": app_key,
        "appSecret": app_secret,
        "tr_id": "TTTC8715R"
        # "tr_cont": "",
        # "custtype": "P"
        
    }

    params = {
        "CANO": acc_no[:8],  # 계좌번호 앞 8지리
        "ACNT_PRDT_CD": acc_no[-2:],  # 계좌번호 뒤 2자리
        "SORT_DVSN": "01",
        "PDNO": "",
        "INQR_STRT_DT": "20230101",
        "INQR_END_DT": "20241231",
        "CBLC_DVSN": "00",
        "CTX_AREA_NK100": "",
        "CTX_AREA_FK100": ""
    }
    res = requests.get(url, headers=headers, params=params)
    body = res.json()
    if 'output1' in body:
        output1_list.extend(body['output1'])
        
    # display(json)
    # print(res.headers['tr_cont'])

    while res.headers['tr_cont'] == "M":
        headers["tr_cont"] = res.headers['tr_cont']
        params["INQR_STRT_DT"] = body['ctx_area_nk100'][:8]
        params["CTX_AREA_NK100"] = body['ctx_area_nk100']
        params["CTX_AREA_FK100"] = body['ctx_area_fk100']
        res = requests.get(url, headers=headers, params=params)
        body = res.json()
        if 'output1' in body:
            output1_list.extend(body['output1'])
        # display(json)
        # print(res.headers['tr_cont'])
    df = pd.DataFrame(output1_list)
    df.rename(columns=column_mapping, inplace=True)
 
    # 각 컬럼에 대해서 쉼표 제거하고 숫자형으로 변환
    for col in cols_to_convert:
        df[col] = df[col].str.replace(',', '')  # 쉼표 제거
        df[col] = pd.to_numeric(df[col], errors='coerce')  # 숫자형으로 변환 (에러가 발생하면 NaN으로 처리)
        
    df['계좌'] = acc_no
    df.loc[df['매도수량'] > 0, '종류'] = '거래소매도'
    df.loc[df['매수수량'] > 0, '종류'] = '거래소매수'
    df.loc[df['매도수량'] > 0, '거래금'] = df['매도금액']
    df.loc[df['매수수량'] > 0, '거래금'] = df['매수금액']
    df.loc[df['매도수량'] > 0, '수량'] = df['매도수량']
    df.loc[df['매수수량'] > 0, '수량'] = df['매수수량']
    df.loc[df['매도수량'] > 0, '정산금'] = df['거래금'] - df['수수료'] - df['세금']
    df.loc[df['매수수량'] > 0, '정산금'] = df['거래금'] + df['수수료'] + df['세금']
    df['거래일'] = pd.to_datetime(df['거래일'], format='%Y%m%d')
    df = df[['거래일', '종류', '거래금', '수량', '잔고', '정산금', '수수료', '세금', '상품명', '코드', '계좌']]
    df.set_index('거래일', inplace=True)
    df.sort_index(inplace=True)
    
    return df

In [36]:
json_0 = get_trade_profit(token_0, api_key_0, api_secret_0, acc_no_0)
json_1 = get_trade_profit(token_1, api_key_1, api_secret_1, acc_no_1)
json_1

,종류,거래금,수량,잔고,정산금,수수료,세금,상품명,코드,계좌
거래일,,,,,,,,,,
2024-04-26,거래소매수,1769500.00,100.00,100,1769574.00,74,0,ACE 미국S&P500,360200,64267316-22
2024-04-26,거래소매수,749500.00,50.00,50,749531.00,31,0,ACE KRX금현물,411060,64267316-22
2024-04-26,거래소매수,24492960.00,233.00,233,24493990.00,1030,0,TIGER KOFR금리액티브(합성),449170,64267316-22
2024-04-26,거래소매수,2152500.00,200.00,200,2152590.00,90,0,SOL 미국배당다우존스(H),452360,64267316-22
2024-04-26,거래소매수,5053850.00,520.00,520,5054062.00,212,0,SOL 미국30년국채커버드콜(합성),473330,64267316-22
...,...,...,...,...,...,...,...,...,...,...
2024-09-03,거래소매도,110150.00,10.00,500,110146.00,4,0,SOL 미국배당다우존스,446720,64267316-22
2024-09-03,거래소매도,554370.00,51.00,300,554347.00,23,0,ACE 미국달러SOFR금리(합성),456880,64267316-22
2024-09-03,거래소매수,893675.00,85.00,85,893712.00,37,0,RISE 글로벌리얼티인컴,475380,64267316-22
